In [4]:
from transformers import pipeline

sentiment_analyser = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
sentiment_analyser(['I have been waiting for a HuggingFace course my whole life.',
                   'I hate this so much!'])

[{'label': 'POSITIVE', 'score': 0.9433633089065552},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

What happens under the hood?

Tokenizer -> Model -> PostProcessing

Tokenizer takes in the raw data in this case the text and converts it into numerical representation for the model.
It does so using the following steps:
- Take input
- Break input down into tokens depending on spaces or punctuation
- Provide the sequence of tokens with a start token and a stop token, the start token for the BERT model is CLS which stands for Classification Tasks and the stop token for said model is SEP which stands for Seperation Tasks.
- Convert all the tokens in the sequence into their numerical representation for the model to ingest.

# Pytorch

### Tokenizer

In [9]:
from transformers import AutoTokenizer

# initialize tokenizer and model from checkpoint name
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# use tokenizer to preprocess inputs:
raw_inputs = [
    'I have been waiting for a HuggingFace course my whole life.',
    'I hate this so much!'
]
inputs_pytorch = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='pt')

In [10]:
print(inputs_pytorch)


{'input_ids': tensor([[  101,  1045,  2031,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])}


### Model

In [14]:
from transformers import AutoModel

# initialize model from checkpoint name
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModel.from_pretrained(checkpoint)

# forward pass
outputs_pytorch = model(**inputs_pytorch)

# print last hidden states of the first batch
print(outputs_pytorch.last_hidden_state.shape)


torch.Size([2, 15, 768])


However, the automodel api will only instantiate once the pre-training head is removed. It will output a high dimensional tensor that is a representation of the sentences passed but not directly useful for classification.

Use Auto model for sequence classification api instead

In [18]:
from transformers import AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs_pytorch = model(**inputs_pytorch)
print(outputs_pytorch.logits)

tensor([[-1.3782,  1.4346],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


It is seen that the outputs are not probabilities yet as each of the outputs don't are from from being between 0 and 1.
This is because each model of the transformers library returns logits.
The logits are converted into probabilities in the third and last step of the pipeline, which is
### Postprocessing 

In [19]:
import torch

'''to convert logits into probabilities we apply the softmax layer'''
predictions = torch.nn.functional.softmax(outputs_pytorch.logits, dim=-1)
print(predictions)

'''the last of these is to see which of these responses 
are positive or negative. 
this is given by id2label field of the model config'''
model.config.id2label

tensor([[5.6636e-02, 9.4336e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


{0: 'NEGATIVE', 1: 'POSITIVE'}

# Tensorflow:

### Tokenizer:

In [21]:
from transformers import AutoTokenizer

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    '''I've been waiting for a HuggingFace course my whole life.''',
    'I hate this so much!'
]

inputs_tensorflow = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors='tf')

In [22]:
print(inputs_tensorflow)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])>}


### Model:

#### AutoModel API

In [23]:
from transformers import TFAutoModel

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model =TFAutoModel.from_pretrained(checkpoint)

outputs_tensorflow = model(inputs_tensorflow)
outputs_tensorflow.last_hidden_state.shape

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


TensorShape([2, 16, 768])

#### AutoModel for Sequence Classification Class:

In [24]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs_tensorflow = model(inputs_tensorflow)
outputs_tensorflow.logits

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-1.5606958,  1.6122808],
       [ 4.169232 , -3.3464477]], dtype=float32)>

### Postprocessing

In [25]:
import tensorflow as tf

predictions = tf.math.softmax(outputs_tensorflow.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.0195428e-02 9.5980465e-01]
 [9.9945587e-01 5.4418371e-04]], shape=(2, 2), dtype=float32)


In [26]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}